In [1]:
import time
from tqdm import tnrange, tqdm
import numpy as np
import secrets
import random 
import os

In [2]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER='SS_VER_2_1'

In [3]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [4]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-rsa-mbedtls-3.3.0
make PLATFORM=$1 CRYPTO_TARGET=MBEDTLS-330 CRYPTO_OPTIONS=RSA OPT=2 SS_VER=$2

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:9-2019-q4-0ubuntu1) 9.2.1 20191025 (release) [ARM/arm-9-branch revision 277599]
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-rsa.c ...
-e Done!
.
Compiling:
-en     simpleserial-rsa-xmega.c ...
-e Done!
.
Compiling:
-en     simpleserial-rsa-arm.c ...
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
-e Done!
.
Compiling:
-en     .././hal/stm32f3/stm32f3_hal.c ...
-e Done!
.
Compiling:
-en     .././hal/stm32f3/stm32f3_hal_lowlevel.c ...
-e Done!
.
Compiling:
-en     .././hal/stm32f3/stm32f3_sysmem.c ...
-e Done!
.
Compiling:


simpleserial-rsa.c: In function 'main':
simpleserial-rsa.c:46:34: warning: passing argument 3 of 'simpleserial_addcmd' from incompatible pointer type [-Wincompatible-pointer-types]
   46 |     simpleserial_addcmd('t', 0,  real_dec);
      |                                  ^~~~~~~~
      |                                  |
      |                                  uint8_t (*)(uint8_t *, uint8_t) {aka unsigned char (*)(unsigned char *, unsigned char)}
In file included from simpleserial-rsa.c:20:
.././simpleserial/simpleserial.h:40:61: note: expected 'uint8_t (*)(uint8_t,  uint8_t,  uint8_t,  uint8_t *)' {aka 'unsigned char (*)(unsigned char,  unsigned char,  unsigned char,  unsigned char *)'} but argument is of type 'uint8_t (*)(uint8_t *, uint8_t)' {aka 'unsigned char (*)(unsigned char *, unsigned char)'}
   40 | int simpleserial_addcmd(char c, unsigned int len, uint8_t (*fp)(uint8_t, uint8_t, uint8_t, uint8_t*));
      |                                                   ~~~~~~~~~~^~~~

In [5]:
fw_path = "../../../hardware/victims/firmware/simpleserial-rsa-mbedtls-3.3.0/simpleserial-rsa-{}.hex".format(PLATFORM)
print(fw_path)
cw.program_target(scope, prog, fw_path)
time.sleep(1)

../../../hardware/victims/firmware/simpleserial-rsa-mbedtls-3.3.0/simpleserial-rsa-CWLITEARM.hex
Detected unknown STM32F ID: 0x446
Extended erase (0x44), this can take ten seconds or more
Attempting to program 55787 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 55787 bytes


In [6]:
scope.clock.adc_mul = 1
scope.adc.stream_mode = True
scope.adc.samples = 10920000
scope.adc.timeout = 20.0
reset_target(scope)

In [7]:
dpdq = int("C1ACF567564274FB07A0BBAD5D26E2983C94D22288ACD763FD8E5600ED4A702DF84198A5F06C2E72236AE490C93F07F83CC559CD27BC2D1CA488811730BB57254959CBF6F8FEF750AEE6977C155579C7D8AAEA56749EA28623272E4F7D0592AF7C1F1313CAC9471B5C523BFE592F517B407A1BD76C164B93DA2D32A383E58357", 16)

proj_path = os.path.join('I:', os.sep, 'TFM', 'updated_lib')
cwd_proj_path = os.path.join(proj_path, "RSA_1024.cwp")
trs_proj_path = os.path.join(proj_path, "RSA_1024.trs")

proj = cw.create_project(cwd_proj_path, overwrite=True)

N = 1
for i in tnrange(N, desc='Capturing traces'):
    target.flush()
    scope.arm()
    plaintext = secrets.token_bytes(128)
    target.simpleserial_write("t", plaintext)

    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    time.sleep(2)
    if SS_VER=='SS_VER_2_1':
        output = target.simpleserial_read_witherrors('r', 128, timeout=10)
    else:
        output = target.simpleserial_read_witherrors('r', 48, timeout=10)
    
    sig = None
    if output['valid']:
        print(output['payload'])
        sig = output['payload']
        
    wave = scope.get_last_trace()

    trace = cw.Trace(wave, plaintext, sig, dpdq.to_bytes(128, 'big'))
    
    
    proj.traces.append(trace)
proj.save()

C:\Users\thor9\CHIPWH~1\cw\home\portable\WPy64-3771\python-3.7.7.amd64\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  # Remove the CWD from sys.path while we load stuff.


Capturing traces:   0%|          | 0/1 [00:00<?, ?it/s]

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:605) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid


USBErrorPipe: LIBUSB_ERROR_PIPE [-9]

In [ ]:
trace.plot()

In [ ]:
scope.dis()

In [ ]:
%%sh -s "$cwd_proj_path" "$trs_proj_path"
cd /c/Users/thor9/github/ot-sca/util
python cw_to_trs.py -i $1 -o $2
